Used for alignment of individual tiles rather than stitched images. Since I've only used this for testing it is somewhat unfinished, but in case alignment of tiles becomes something you'd want to do it should be very little work to adjust this notebook for that.

In [ ]:
import SimpleITK as sitk
from pathlib import Path
from skimage import io
import numpy as np
from datetime import datetime
from skimage import img_as_uint

In [30]:
path_plate = Path(r"/links/groups/treutlein/DATA/imaging/PW/4i/plate14")
path_results = Path(r"/links/groups/treutlein/USERS/pascal_noser/plate14_results/alignment/tile_alignment_name")

# Create list with all the cycle folder names in the right order
cycles = ["cycle0",
         "cycle1",
         "cycle2",
         "cycle3",
         "cycle4",
         "cycle5",
         "cycle5_0",
         "cycle6",
         "cycle7",
         "cycle8",
         "cycle9",
         "cycle10",
         "cycle10_0",
         "cycle11",
         "cycle12",
         "cycle13",
         "cycle14",
         "cycle15",
         "cycle15_0",
         "cycle16",
         "cycle17",
         "cycle18",
         "cycle19",
         "cycle20",
         "cycle20_0",
         "cycle21",
         "cycle1_2",
         "cycle1_3"]

# Define points to be aligned
points = ["Point0006"]

In [23]:
tiles_dict = {
#    "Point0006": ["0002", "0003", "0004", "0005",
#                 "0006", "0007", "0008", "0009", "0010",
#                 "0013", "0014", "0015", "0016", "0017",
#                 "0018", "0019", "0020", "0021", "0022",
#                 "0026", "0027", "0028", "0029",
#                 "0031", "0032", "0033"]
    "Point0006": ["0007", "0008"]
}

In [ ]:
# Initialise elastix and transformix
elastix_filter = sitk.ElastixImageFilter()
elastix_filter.SetParameterMap(sitk.ReadParameterFile(r"Elastix/param_maps_tiles/rigid.txt"))
elastix_filter.AddParameterMap(sitk.ReadParameterFile(r"Elastix/param_maps_tiles/bspline.txt"))

transformix_filter = sitk.TransformixImageFilter()

for point in points:
    print("\n################################")
    print("################################")
    print("Starting alignment of", point)
    print("################################")
    print("################################")
    
    cycles_local = cycles.copy()

    # Define first reference cycle. Ideally cycle1, else cycle2 and else cycle3 (assuming cycle3 is present in all IDs)
    if "cycle1" in cycles_local:
        first_ref = "cycle1"
    elif "cycle2" in cycles_local:
        first_ref = "cycle2"
    else:
        first_ref = "cycle3"
    
    print("First reference: ", first_ref)
    reference_cycles = [first_ref, "cycle6", "cycle11", "cycle16", "cycle21"]
    
    # remove first_ref from cycles list because that one won't need to be aligned
    cycles_local.remove(first_ref)
    
    # Get relevant tiles
    tiles = tiles_dict[point]
    
    for tile in tiles:
        print("\n--------------------------------")
        print("Tile:", tile)
        print("--------------------------------")
        start_timer_tile = datetime.now()
        # Specify name of file to be loaded
        filename = "multichannel_" + point + "_Point" + tile + "_ChannelSD 640,SD 488.tif"
        
        tile_name = "Tile"+tile
        
        # Create output directory if it doesn't exist. Doesn't overwrite existing directories
        Path(path_results/point/tile_name/first_ref).mkdir(parents=True, exist_ok=True)
        
        # Save first ref image unaltered. Already load as fixed_img since it will be used as such for the first cycles
        fixed_img_name = point+"_"+tile_name+"_"+first_ref+".tif"
        fixed_img = io.imread(str(path_plate/first_ref/"multichannel"/filename))
        # Create mask for the fixed img
        fixed_mask = img_as_uint(fixed_img[...,2]>0)
        if not Path(path_results/point/tile_name/first_ref/fixed_img_name).is_file():
            io.imsave(str(path_results/point/tile_name/first_ref/fixed_img_name), fixed_img, check_contrast=False)
            
        for cycle in cycles_local:
            print("Aligning", cycle)
            # Specify output path
            out_path = path_results/point/tile_name/cycle
            
            # If directory already exists don't do the alignment
            if out_path.is_dir():
                print("Directory already exists. Skipping", cycle)
            else:
                print("Reference image:", fixed_img_name)
                # Create directory
                Path(out_path/"param_maps").mkdir(parents=True, exist_ok=True)
                
                # Load moving image
                moving_img = io.imread(str(path_plate/cycle/"multichannel"/filename))

                # Elastix
                start=datetime.now()
                elastix_filter.SetFixedImage(sitk.GetImageFromArray(fixed_img[...,2]))
                elastix_filter.SetFixedMask(sitk.Cast(sitk.GetImageFromArray(fixed_mask), sitk.sitkUInt8))
                elastix_filter.SetMovingImage(sitk.GetImageFromArray(moving_img[...,2]))
                elastix_filter.SetOutputDirectory(str(out_path/"param_maps"))
                #elastix_filter.LogToConsoleOn()
                elastix_filter.Execute()
                
                # Transformix
                channels = []
                for channel in range(moving_img.shape[2]):            
                    transformix_filter.SetTransformParameterMap(elastix_filter.GetTransformParameterMap())
                    transformix_filter.SetMovingImage(sitk.GetImageFromArray(moving_img[..., channel]))
                    channel_aligned = transformix_filter.Execute()
                    
                    # Convert to numpy array
                    channel_aligned = sitk.GetArrayFromImage(channel_aligned)
                    # Cap values just in case
                    channel_aligned[channel_aligned < 0] = 0
                    channel_aligned[channel_aligned > 65535] = 65535
                    # Convert to uint16
                    channel_aligned = channel_aligned.astype(np.uint16)
                    # append aligned channel to the list
                    channels.append(channel_aligned)
                
                # combine channels into an image
                img_aligned = np.dstack(channels)
                
                # save aligned image
                img_aligned_name = point+"_"+tile_name+"_"+cycle+".tif"
                io.imsave(str(out_path/img_aligned_name), img_aligned, check_contrast=False)
                
                print("Alignment of cycle took ", datetime.now()-start)
                
                # set new reference cycle if necessary
                if cycle in reference_cycles:
                    ref_cycle = cycle
                    print("Setting reference cycle to", ref_cycle)
                    
                    # Specify new fixed image
                    fixed_img_name = point+"_"+tile_name+"_"+ref_cycle+".tif"
                    fixed_img = io.imread(str(path_results/point/tile_name/ref_cycle/fixed_img_name))
                    fixed_mask = img_as_uint(fixed_img[...,2]>0)
        
        print("Alignment of all cycles in this tile took ", datetime.now()-start_timer_tile)


################################
################################
Starting alignment of Point0006
################################
################################
First reference:  cycle1

--------------------------------
Tile: 0007
--------------------------------
Aligning cycle0
Reference image: Point0006_Tile0007_cycle1.tif
Alignment of cycle took  0:03:44.892624
Aligning cycle2
Reference image: Point0006_Tile0007_cycle1.tif
Alignment of cycle took  0:03:45.759302
Aligning cycle3
Reference image: Point0006_Tile0007_cycle1.tif
Alignment of cycle took  0:03:49.285257
Aligning cycle4
Reference image: Point0006_Tile0007_cycle1.tif
Alignment of cycle took  0:03:45.781345
Aligning cycle5
Reference image: Point0006_Tile0007_cycle1.tif
Alignment of cycle took  0:03:50.043300
Aligning cycle5_0
Reference image: Point0006_Tile0007_cycle1.tif
Alignment of cycle took  0:03:48.516467
Aligning cycle6
Reference image: Point0006_Tile0007_cycle1.tif
Alignment of cycle took  0:03:47.458918
Setting r